# AWS Bedrock Integration

This notebook aims to provide an example of how to access [Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html) and call one of their hosted LLMs.

The LLM of this example is an Anthropic Claude model, and we are going to simulate a virtual assistant.

## Pre-requisites

In order to interact with AWS Bedrock, you need to configure authentication first. 

Follow [these instructions](https://github.com/elastic/cloud/blob/master/wiki/AWS.md#configuring-okta-awscli-for-cli--api-access) in order to do so. You will need a Yubikey to enable MFA, Touch ID in MacOS no longer works.

## Example

The first thing we need to do is to create the Bedrock runtime client. To do so, we need to authenticate ourselves in the ML account:

In [1]:
import boto3

ml_session = boto3.Session(profile_name="ml", region_name="us-east-1")
bedrock_runtime_client = ml_session.client("bedrock-runtime")

Let's define now some initial parameters:

In [2]:
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
system_prompt = "Please respond only with emoji."
max_tokens = 1000

Let's invoke the model with an initial prompt:

In [3]:
user_message = {"role": "user", "content": "Hello there! How are you doing today?"}
messages = [user_message]

In [4]:
import json

from botocore.exceptions import ClientError

try:
    body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": max_tokens,
            "system": system_prompt,
            "messages": messages,
        }
    )

    response = bedrock_runtime_client.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response["body"].read())
    print(response_body)

except ClientError as e:
    message = e.response["Error"]["Message"]
    print(f"A client error occurred: {message}")

{'id': 'msg_bdrk_01FSqdzLSLoeVD7c3ucTDy3q', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-sonnet-20240229', 'content': [{'type': 'text', 'text': '👋😊 (Waving hello with a smiling face)'}], 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 22, 'output_tokens': 20}}


To check the different models and options that you have at hand with Bedrock, check [this chapter](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters.html) of the documentation.

To check the pricing of each inference, see the [pricing information](https://aws.amazon.com/bedrock/pricing/).

Check [here](https://docs.aws.amazon.com/bedrock/latest/userguide/models-regions.html) the models supported by region.

And, finally, check [here](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html) the Converse API, which is more intended for conversations.

## Optional: Store the result of MLflow

You can use MLflow to track your prompts and responses.

First, connect to our centralized MLflow server:

In [5]:
ml_sagemaker_session = boto3.Session(
    profile_name="sagemaker"
)  # you can also use the existing ml session

In [6]:
import os

credentials = ml_sagemaker_session.get_credentials().get_frozen_credentials()
os.environ["AWS_ACCESS_KEY_ID"] = credentials.access_key
os.environ["AWS_SECRET_ACCESS_KEY"] = credentials.secret_key
os.environ["AWS_SESSION_TOKEN"] = credentials.token

In [7]:
import mlflow

mlflow_server_arn = (
    "arn:aws:sagemaker:us-east-1:879381254630:mlflow-tracking-server/ml-rd-mlflow-server"
)

mlflow.set_tracking_uri(mlflow_server_arn)

Now, let's log the input prompt and the response we got:

In [8]:
mlflow.set_experiment("bedrock-example")

<Experiment: artifact_location='s3://ml-rd-mlflow-artifact-storage/artifacts/35', creation_time=1734091856180, experiment_id='35', last_update_time=1734091856180, lifecycle_stage='active', name='bedrock-example', tags={}>

In [10]:
with mlflow.start_run(run_name="bedrock-emoji-example"):
    mlflow.log_text(text=system_prompt, artifact_file="system_prompt.txt")
    mlflow.log_text(text=json.dumps(user_message), artifact_file="user_message.json")
    mlflow.log_text(text=json.dumps(response_body), artifact_file="response.json")

2024/12/13 13:14:18 INFO mlflow.tracking._tracking_service.client: 🏃 View run bedrock-emoji-example at: https://us-east-1.experiments.sagemaker.aws/#/experiments/35/runs/2cfc68b3bcec44bf9018e19e2603a50d.
2024/12/13 13:14:18 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://us-east-1.experiments.sagemaker.aws/#/experiments/35.


Now go and check the results of the mlflow UI!